In [1]:
import json
import os
import re

from easydict import EasyDict as edict
import requests as req
import pymysql as sql
import redis

In [2]:
SEP       = os.path.sep
ROOT_PATH = SEP.join(os.getcwd().split(SEP)[:-5])
JSON_PATH = f'{ROOT_PATH}/utils/configs'

read_json = lambda path: edict(json.loads(open(path, 'r').read()))
REPLACES  = ['<br>', '<strong>*</strong>', '음성이 지원되는 언어']
CONFIGS   = read_json(f'{JSON_PATH}/config.json')
PORTS     = read_json(f'{JSON_PATH}/ports.json')
URLS      = {'appdetail' : 'https://store.steampowered.com/api/appdetails?appids'}

CONFIGS   = edict(CONFIGS)
PORTS     = edict(PORTS)

In [3]:
passwd = CONFIGS.sql_passwd
rport  = PORTS.redis_port
host   = CONFIGS.global_host
port   = PORTS.sql_port
user   = CONFIGS.sql_user

conn   = sql.connect(host = host, user = user, port = port, 
                     password = passwd, db = 'DoveNest')
rconn  = redis.StrictRedis(host = host, port = rport, db = 1)
cursor = conn.cursor()

In [4]:
datas  = rconn.scan()

In [5]:
for data in datas[1]:

    try:
        data      = json.loads(rconn.get(data))
        data      = data['data']
        name      = data['name']
        appid     = data['steam_appid']
        thumbnail = data['header_image']
        genres    = ', '.join(d['description'] for d in data['genres'])
        
        if data['is_free']: price = '₩ 0'
        else: price = data['price_overview']['final_formatted']

        developers = ', '.join(data['developers'])
        publishers = ', '.join(data['publishers'])
        languages  = data['supported_languages']
        
        for replace in REPLACES: languages = languages.replace(replace, '')
        platforms  = data['platforms']

        if data['release_date']['coming_soon']: date = 'N/A'
        else: date = data['release_date']['date']

        print(platforms)
    except Exception as e:
        print(data['name'], e)

    finally: print('\n\n')


Call of Duty 4: Modern Warfare 'data'



{'windows': True, 'mac': False, 'linux': False}



{'windows': True, 'mac': False, 'linux': False}



{'windows': True, 'mac': False, 'linux': True}



{'windows': True, 'mac': True, 'linux': True}



{'windows': True, 'mac': False, 'linux': False}



{'windows': True, 'mac': False, 'linux': False}



{'windows': True, 'mac': False, 'linux': False}



{'windows': True, 'mac': False, 'linux': False}



{'windows': True, 'mac': False, 'linux': False}





In [6]:
len(datas[1])

10